In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dnyaneshwarkhaire","key":"f8f3454bc4c4839643189f19bad4bf19"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d gauravduttakiit/agricultural-pests-dataset
# !kaggle datasets download -d salader/dogs-vs-cats

 96% 98.0M/102M [00:01<00:00, 115MB/s]
100% 102M/102M [00:01<00:00, 85.9MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/agricultural-pests-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
# Import required libraries
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [ ]:
# Load the pre-trained model and set requires_grad to False
model_conv = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 102MB/s]


In [ ]:
for param in model_conv.parameters():
    param.requires_grad = False

# Define a forward hook to extract the output of the desired layer
features = {}
def hook(module, input, output):
    pooled_output = torch.nn.functional.avg_pool2d(output, kernel_size=7)
    features['output'] = pooled_output.view(pooled_output.size(0), -1).detach().numpy()
model_conv.layer4.register_forward_hook(hook)

# Define the data transformation and load the data
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Give path of data set
data_dir = '/content'
image_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms)
data_loader = torch.utils.data.DataLoader(image_dataset, batch_size=32, shuffle=False, num_workers=4)

#https://audhiaprilliant.medium.com/fixed-feature-extractor-as-the-transfer-learning-method-for-image-classification-using-mobilenet-b26376e25d49
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Pass the input images through the model and extract the features
all_features = []
for inputs, labels in data_loader:
    outputs = model_conv(inputs)
    features_array = features['output']
    all_features.append(features_array)
all_features = np.concatenate(all_features, axis=0)

# Pass validation data through the model and extract the features
val_features = []
for inputs, labels in val_loader:
    outputs = model_conv(inputs)
    features_array = features['output']
    val_features.append(features_array)
val_features = np.concatenate(val_features, axis=0) # All above codes are refered from the link given in the assignment itself.

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
# Stored all data with suitable variable name
X_train = all_features
X_test = val_features
y_train = image_dataset.targets
y_test = val_dataset.targets

In [ ]:
X_train.shape

(4395, 512)

In [ ]:
# Now in similar way training model using gridsearchCV and classifying the images
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score
# Define the parameter grid to search over
param_grid_rbf = {'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1, 1]}

# Create an RBF SVM model for the binary classification problem
svm_model_binary_rbf = SVC(kernel='rbf')

# Use GridSearchCV to search over the parameter grid and perform 5-fold cross-validation for the binary classification problem
grid_search_binary_rbf = GridSearchCV(svm_model_binary_rbf, param_grid_rbf, cv=5)

# Fit the model using the training data for binary classification problem
grid_search_binary_rbf.fit(X_train, y_train)

# Print the best hyperparameters found by GridSearchCV for the binary classification problem
print('Best hyperparameters for binary classification (RBF SVM):', grid_search_binary_rbf.best_params_)

# Use the best model found by GridSearchCV for the binary classification problem to make predictions on the test data
y_pred_binary_rbf = grid_search_binary_rbf.predict(X_test)

# Calculate evaluation metrics
f1 = f1_score(y_test, y_pred_binary_rbf, average='weighted')
accuracy = accuracy_score(y_test, y_pred_binary_rbf)
# Print the results
print('F1 score: {:.4f}'.format(f1))
print('accuracy: {:.4f}'.format(accuracy))

Best hyperparameters for binary classification (RBF SVM): {'C': 10, 'gamma': 0.001}
F1 score: 0.5214
accuracy: 0.5205
